In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

!pip install -U efficientnet

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.io as sio
import os
import sys

from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.layers import Input
from keras import callbacks,optimizers
from keras.callbacks import LearningRateScheduler

from sklearn.metrics import classification_report, confusion_matrix
import pickle
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
import os
from shutil import copyfile
import cv2
from keras.preprocessing import image
from keras.preprocessing.image import array_to_img, img_to_array, load_img,save_img
import numpy as np
from keras.utils import to_categorical
from keras.models import load_model
import math

mainFolder = "/gdrive/My Drive/glaucoma"
os.chdir(mainFolder)

TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


In [ ]:
imageFolder = "Data"
subFolders = ["non_glaucoma","suspicious_glaucoma"]
classes = {'non_glaucoma':0, 'suspicious_glaucoma':1}

fileList = []
Y = []

for index,folder in enumerate(subFolders):
    files = os.listdir(os.path.join(imageFolder,folder))
    for file in files:
      fileList.append(os.path.join(folder,file))
    for i in range(len(files)):
      Y.append(classes[os.path.split(folder)[-1]])

Y = np.array(Y)
fileList = np.array(fileList)

In [ ]:
def FileRead(imageFolder,fileList,imageSize):
    X = []
    for file in fileList:
      img = image.load_img(os.path.join(imageFolder,file), target_size=imageSize)
      img = image.img_to_array(img)
      X.append(img)
    return np.array(X)

trainFiles,testFiles,y_train,y_test = train_test_split(fileList,to_categorical(Y), test_size=0.20, random_state=1,stratify=to_categorical(Y))
trainFiles,validationFiles,y_train,y_val = train_test_split(trainFiles,y_train, test_size=0.10, random_state=1,stratify=y_train)

# VGG16

In [ ]:

from keras.applications.vgg16  import VGG16
from keras.applications.vgg16 import preprocess_input

x_train = preprocess_input(FileRead(imageFolder,trainFiles,(224,224)))
x_test = preprocess_input(FileRead(imageFolder,testFiles,(224,224)))
x_val = preprocess_input(FileRead(imageFolder,validationFiles,(224,224)))

resultFolder = "Results/VGG16"

checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = VGG16(weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,
                    batch_size=64,
                    validation_data=(x_val, y_val),
                    verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 5s 0us/step

Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 78s 22ms/step - loss: 0.5476 - accuracy: 0.7467 - val_loss: 0.3537 - val_accuracy: 0.8329

Epoch 00001: val_loss improved from inf to 0.35367, saving model to Results/VGG16/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 61s 18ms/step - loss: 0.2857 - accuracy: 0.8729 - val_loss: 0.2169 - val_accuracy: 0.9049

Epoch 00002: val_loss improved from 0.35367 to 0.21695, saving model to Results/VGG16/model.hdf5
Epoch 3/50
3494/3494 [==============================] - 62s 18ms/step - loss: 0.1524 - accuracy: 0.9405 - val_loss: 0.1833 - val_accuracy: 0.9332

Epoch 00003: val_loss improved from 0.21695 to 0.18326, saving model to Results/VGG16/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 62s 18ms/step - loss:

# EfficientNET

In [ ]:
import efficientnet.keras as efn 

os.chdir(mainFolder)

x_train = efn.preprocess_input(FileRead(imageFolder,trainFiles,(224,224)))
x_test = efn.preprocess_input(FileRead(imageFolder,testFiles,(224,224)))
x_val = efn.preprocess_input(FileRead(imageFolder,validationFiles,(224,224)))

resultFolder ="Results/EfficientNet" 

checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = efn.EfficientNetB0(weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,
                    batch_size=32,
                    validation_data=(x_val, y_val),
                     verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)

16809984/16804768 [==============================] - 3s 0us/step
Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 67s 19ms/step - loss: 0.3902 - accuracy: 0.8246 - val_loss: 0.3359 - val_accuracy: 0.8509

Epoch 00001: val_loss improved from inf to 0.33590, saving model to Results/EfficientNet/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 56s 16ms/step - loss: 0.1924 - accuracy: 0.9305 - val_loss: 0.2241 - val_accuracy: 0.9203

Epoch 00002: val_loss improved from 0.33590 to 0.22414, saving model to Results/EfficientNet/model.hdf5
Epoch 3/50
3494/3494 [==============================] - 55s 16ms/step - loss: 0.1234 - accuracy: 0.9559 - val_loss: 0.1654 - val_accuracy: 0.9434

Epoch 00003: val_loss improved from 0.22414 to 0.16539, saving model to Results/EfficientNet/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 56s 16ms/step - loss: 0.0796 - accuracy: 0.9720 - val_loss: 0.1468 - val_accuracy: 

# denseNET

In [ ]:
from keras.applications.densenet  import DenseNet121
from keras.applications.densenet import preprocess_input

os.chdir(mainFolder)

x_train = preprocess_input(FileRead(imageFolder,trainFiles,(224,224)))
x_test = preprocess_input(FileRead(imageFolder,testFiles,(224,224)))
x_val = preprocess_input(FileRead(imageFolder,validationFiles,(224,224)))

resultFolder ="Results/DenseNET" 


checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = DenseNet121(weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])



history = model.fit(x_train,y_train,
                    batch_size=16,
                    validation_data=(x_val, y_val),
                     verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)

Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 99s 28ms/step - loss: 0.2601 - accuracy: 0.8864 - val_loss: 0.2619 - val_accuracy: 0.8895

Epoch 00001: val_loss improved from inf to 0.26192, saving model to Results/DenseNET/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 75s 21ms/step - loss: 0.0838 - accuracy: 0.9728 - val_loss: 0.1672 - val_accuracy: 0.9306

Epoch 00002: val_loss improved from 0.26192 to 0.16720, saving model to Results/DenseNET/model.hdf5
Epoch 3/50
3494/3494 [==============================] - 75s 21ms/step - loss: 0.0371 - accuracy: 0.9894 - val_loss: 0.1623 - val_accuracy: 0.9589

Epoch 00003: val_loss improved from 0.16720 to 0.16233, saving model to Results/DenseNET/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 75s 21ms/step - loss: 0.0202 - accuracy: 0.9943 - val_loss: 0.1019 - val_accuracy: 0.9589

Epoch 00004: val_loss improved from 0.16233 to 0.10187, saving model 

# MobilNET

In [ ]:
from keras.applications.mobilenet  import MobileNet
from keras.applications.mobilenet import preprocess_input

os.chdir(mainFolder)

x_train = preprocess_input(FileRead(imageFolder,trainFiles,(224,224)))
x_test = preprocess_input(FileRead(imageFolder,testFiles,(224,224)))
x_val = preprocess_input(FileRead(imageFolder,validationFiles,(224,224)))

resultFolder ="Results/MobilNET" 


checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = MobileNet(weights='imagenet',include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(x_train,y_train,
                    batch_size=64,
                    validation_data=(x_val, y_val),
                     verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 3s 0us/step

Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 42s 12ms/step - loss: 0.3541 - accuracy: 0.8366 - val_loss: 0.5041 - val_accuracy: 0.7506

Epoch 00001: val_loss improved from inf to 0.50406, saving model to Results/MobilNET/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 31s 9ms/step - loss: 0.1126 - accuracy: 0.9628 - val_loss: 0.7384 - val_accuracy: 0.6607

Epoch 00002: val_loss did not improve from 0.50406
Epoch 3/50
3494/3494 [==============================] - 31s 9ms/step - loss: 0.0521 - accuracy: 0.9926 - val_loss: 0.4851 - val_accuracy: 0.7918

Epoch 00003: val_loss improved from 0.50406 to 0.48509, saving model to Results/MobilNET/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 31s 9ms/step - loss: 0.0272 - accuracy: 0.9991 - val_loss: 0.3895 - val_accuracy: 0.8458

Epoch 00004: val_loss improved from 0.48509 to 0.38954,

# ResNET

In [ ]:
from keras.applications.resnet  import ResNet50
from keras.applications.resnet import preprocess_input

os.chdir(mainFolder)

x_train = preprocess_input(FileRead(imageFolder,trainFiles,(224,224)))
x_test = preprocess_input(FileRead(imageFolder,testFiles,(224,224)))
x_val = preprocess_input(FileRead(imageFolder,validationFiles,(224,224)))

resultFolder ="Results/ResNET" 



checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = ResNet50(weights='imagenet',include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,
                    batch_size=32,
                    validation_data=(x_val, y_val),
                     verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)


94773248/94765736 [==============================] - 8s 0us/step
Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 74s 21ms/step - loss: 0.4576 - accuracy: 0.8263 - val_loss: 33.3249 - val_accuracy: 0.3548

Epoch 00001: val_loss improved from inf to 33.32488, saving model to Results/ResNET/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 64s 18ms/step - loss: 0.2567 - accuracy: 0.8947 - val_loss: 1.7083 - val_accuracy: 0.7352

Epoch 00002: val_loss improved from 33.32488 to 1.70833, saving model to Results/ResNET/model.hdf5
Epoch 3/50
3494/3494 [==============================] - 64s 18ms/step - loss: 0.1912 - accuracy: 0.9230 - val_loss: 0.3546 - val_accuracy: 0.8638

Epoch 00003: val_loss improved from 1.70833 to 0.35456, saving model to Results/ResNET/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 64s 18ms/step - loss: 0.1553 - accuracy: 0.9359 - val_loss: 1.0439 - val_accuracy: 0.7789

Epoch 

# Inception V3

In [ ]:
from keras.applications.inception_v3  import InceptionV3
from keras.applications.inception_v3 import preprocess_input

os.chdir(mainFolder)

x_train = preprocess_input(FileRead(imageFolder,trainFiles,(299,299)))
x_test = preprocess_input(FileRead(imageFolder,testFiles,(299,299)))
x_val = preprocess_input(FileRead(imageFolder,validationFiles,(299,299)))

resultFolder ="Results/InceptionV3" 

checkpoint = ModelCheckpoint(os.path.join(resultFolder,"model.hdf5"), monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

base_model = InceptionV3(weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = True  

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train,y_train,
                    batch_size=32,
                    validation_data=(x_val, y_val),
                     verbose=1,epochs=50,
                    callbacks=[checkpoint]
                    )

model = load_model(os.path.join(resultFolder,"model.hdf5"))
y_head = model.predict(x_test)
clfReport = classification_report(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1) ,digits=4)
cnfMatris = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_head,axis=1))
print(cnfMatris)
print(clfReport)

np.save(os.path.join(resultFolder,"confMatris.npy"),cnfMatris)
with open(os.path.join(resultFolder,'trainHistoryDict'), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
with open(os.path.join(resultFolder,'clfReport'), 'wb') as file_pi:
        pickle.dump(clfReport, file_pi)        
np.save(os.path.join(resultFolder,"preds.npy"),y_head)
np.save(os.path.join(resultFolder,"y_test.npy"),y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Train on 3494 samples, validate on 389 samples
Epoch 1/50
3494/3494 [==============================] - 96s 28ms/step - loss: 0.3538 - accuracy: 0.8457 - val_loss: 4.7662 - val_accuracy: 0.3805

Epoch 00001: val_loss improved from inf to 4.76618, saving model to Results/InceptionV3/model.hdf5
Epoch 2/50
3494/3494 [==============================] - 74s 21ms/step - loss: 0.1915 - accuracy: 0.9282 - val_loss: 1.3850 - val_accuracy: 0.6607

Epoch 00002: val_loss improved from 4.76618 to 1.38496, saving model to Results/InceptionV3/model.hdf5
Epoch 3/50
3494/3494 [==============================] - 74s 21ms/step - loss: 0.1397 - accuracy: 0.9471 - val_loss: 0.2673 - val_accuracy: 0.9100

Epoch 00003: val_loss improved from 1.38496 to 0.26727, saving model to Results/InceptionV3/model.hdf5
Epoch 4/50
3494/3494 [==============================] - 74s 21ms/step - loss: 0.1238 - accuracy: 0.9508 - val_loss: 0.2884 